# SARIMAX model

In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
import pmdarima as pm
from pmdarima.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAXResults, SARIMAX
import matplotlib.pyplot as plt


from modelling import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

In [17]:
# Set file path and parameters
file_path = "../../data/fulldata.csv"

In [18]:

loader = SARIMADataLoader(file_path, 
                    use_temp_pca=True, 
                    use_wind_pca=True, 
                    use_fourier=True, 
                    use_oil=True, 
                    use_gas=True)

loader.load_data()
spot_price, exog_data = loader.preprocess_data()

## Running LASSO

In [19]:
# standardize features
if not exog_data.empty:
    scaler = StandardScaler()
    exog_scaled = scaler.fit_transform(exog_data)
else:
    exog_scaled = None
    print("No exogenous variables selected. Skipping scaling step.")


In [20]:
# Step 3: Fit LASSO Regression
print("Fitting LASSO regression for feature selection...")
lasso = LassoCV(cv=5, max_iter=5000)  # Cross-validation to find the best alpha
lasso.fit(exog_scaled, spot_price)

Fitting LASSO regression for feature selection...


LassoCV(cv=5, max_iter=5000)

## SARIMAX model

### Preparing data for SARIMAX

In [21]:
# Step 4: Extract Selected Features
selected_features = exog_data.columns[lasso.coef_ != 0]
print("Selected Features:", selected_features)

# Reduce exogenous data to selected features
exog_selected = exog_data[selected_features]

Selected Features: Index(['temp_pca_1', 'temp_pca_2', 'temp_pca_3', 'wind_speed_pca_1',
       'wind_speed_pca_2', 'wind_speed_pca_3', 'wind_speed_pca_4',
       'wind_speed_pca_5', 'wind_speed_pca_6', 'wind_speed_pca_7',
       'wind_speed_pca_8', 'wind_speed_pca_9', 'wind_speed_pca_10',
       'wind_speed_pca_11', 'wind_speed_pca_12', 'wind_speed_pca_13',
       'wind_speed_pca_14', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
       'month_sin', 'month_cos', 'oil_price', 'gas_price'],
      dtype='object')


In [22]:
# Define the date range for training
train_start = '2017-10-27'
train_end = '2024-07-31'

# Filter the spot_price and exog_selected based on the date range
train_data = spot_price.loc[train_start:train_end].asfreq('h')
train_exog = exog_selected.loc[train_data.index].asfreq('h')

# Confirm the shape of the data
print("Training data shape:", train_data.shape)
#print("Training exogenous features shape:", train_exog.shape)


Training data shape: (59280,)


### Running AUTO ARIMA on subset of the data

In [23]:
# define the date range for training
# train_start = '2023-10-27'
# train_end = '2024-07-31'

# # filter the spot_price and exog_features based on the date range
# train_data = spot_price.loc[train_start:train_end]
# train_exog = exog_selected.loc[train_start:train_end]

# # split into train and test (optional)
# train_y, test_y = train_test_split(train_data, test_size=0.1)


# # define exogenous variables and ensure alignment with the target variable
# exog_features = exog_data.loc[train_data.index]

# # run auto_arima with exogenous variables
# auto_model = pm.auto_arima(
#    train_data,                 # Target variable (spot_price)
#    exogenous=exog_features,    # Exogenous predictors
#    seasonal=True,              # Allows seasonal terms
#    m=12,                       # Seasonality frequency (12 for monthly, 24 for daily hours)
#    stepwise=True,              # Stepwise search for efficiency
#    trace=True,                 # Print progress
#    suppress_warnings=True,     # Ignore warnings
#    error_action='ignore',      # Ignore invalid models
#    max_order=(5, 2)            # Limit AR, MA, and seasonal orders for faster search
# )

# print(auto_model.summary())



### Training SARIMAX model

In [24]:
#After auto_arima is done
order = (5,1,0)       # (p, d, q)
seasonal_order = (2,0,0,12)  # (P, D, Q, m)

In [25]:
model = SARIMAX(
            train_data,
            exog=train_exog,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=True,
            enforce_invertibility=True,
        )

model = model.fit(disp=True)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           33     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.02300D+00    |proj g|=  1.55383D-01


 This problem is unconstrained.



At iterate    1    f=  6.00293D+00    |proj g|=  3.70600D-02

At iterate    2    f=  5.99814D+00    |proj g|=  1.60099D-02

At iterate    3    f=  5.99770D+00    |proj g|=  7.42761D-03

At iterate    4    f=  5.99765D+00    |proj g|=  2.49226D-03

At iterate    5    f=  5.99763D+00    |proj g|=  1.46774D-03

At iterate    6    f=  5.99762D+00    |proj g|=  1.46182D-03

At iterate    7    f=  5.99758D+00    |proj g|=  3.23433D-03

At iterate    8    f=  5.99748D+00    |proj g|=  6.68904D-03

At iterate    9    f=  5.99720D+00    |proj g|=  1.23768D-02

At iterate   10    f=  5.99659D+00    |proj g|=  1.95357D-02

At iterate   11    f=  5.99563D+00    |proj g|=  2.51337D-02

At iterate   12    f=  5.99444D+00    |proj g|=  2.28914D-02

At iterate   13    f=  5.99363D+00    |proj g|=  9.80233D-03

At iterate   14    f=  5.99344D+00    |proj g|=  4.89333D-03

At iterate   15    f=  5.99341D+00    |proj g|=  1.08934D-03

At iterate   16    f=  5.99341D+00    |proj g|=  1.04462D-03

At iter

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



At iterate   50    f=  5.98870D+00    |proj g|=  2.67538D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   33     50     53      1     0     0   2.675D-03   5.989D+00
  F =   5.9887024148800290     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


### Saving model for future use

In [26]:
# save the model
model.save('/Users/johan/Documents/04 Uni/09 Asset Pricing Data/SARMIA(X)/sarimax_model.pkl')

In [27]:
# load the model
model = SARIMAXResults.load('/Users/johan/Documents/04 Uni/09 Asset Pricing Data/SARMIA(X)/sarimax_model.pkl')

### Forecasting

In [28]:
# define the forecast start and end
forecast_start = pd.Timestamp('2024-08-01 00:00:00')
forecast_end = pd.Timestamp('2024-11-29 23:00:00')  # Shorter period for demonstration

# create a daily index for the days we want to forecast
daily_forecast_start_times = pd.date_range(start=forecast_start, end=forecast_end, freq='D', inclusive='both')

# actual_data includes actual values over the entire period (for demonstration)
actual_data = spot_price.loc[:forecast_end]

# ensure exogenous data coverage for the full test period + 24 hours before
full_exog_index = pd.date_range(start=forecast_start - pd.Timedelta(hours=24),
                                end=forecast_end, freq='h')

exog_features_reduced = exog_selected.reindex(full_exog_index)
# shift exogenous features by 24 hours for forecasting
exog_features_shifted = exog_features_reduced.shift(24)

# extract original exogenous columns from the loaded model
orig_exog_columns = model.data.orig_exog.columns
forecasts = pd.DataFrame()
all_forecasts = []
dat_start = model.data.orig_endog.index[-1]

for start_time in daily_forecast_start_times:
    # forecast 24 hours from start_time
    end_time = start_time + pd.Timedelta(hours=23)

    # data known up to 24 hours before the forecast day starts
    dat_end = end_time - pd.Timedelta(hours=24)
    new_dat = actual_data.loc[(dat_start + pd.Timedelta(hours=1)):dat_end]

    # if any new data is available, append it w. exogenous variable to the model
    if not new_dat.empty:
        new_exog = exog_features_reduced.loc[new_dat.index, orig_exog_columns]
        model = model.append(new_dat, exog=new_exog, refit=False)
        dat_start = dat_end

    # forecast for the next 24 hours
    forecast_index = pd.date_range(start=start_time, periods=24, freq='h')

    # give 24h previous exogenous data to predict from
    exog_forecast = exog_features_shifted.loc[start_time:end_time, orig_exog_columns]

    # Get a 24-step forecast
    forecast_result = model.get_forecast(steps=24, exog=exog_forecast)
    day_forecasts = forecast_result.predicted_mean
    all_forecasts.append(day_forecasts)

# combine all daily forecasts
forecasts = pd.concat(all_forecasts)

In [29]:
# evaluation of forecasts
actuals = spot_price[spot_price.index >= forecast_start]

mae = mean_absolute_error(actuals, forecasts)
rmse = root_mean_squared_error(actuals, forecasts)
print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}")


MAE: 224.6852, RMSE: 313.6268


In [30]:
forecast_array = forecasts.values
np.save('output/forecast_sarimax.npy', forecast_array)